In [1]:
import os,sys
 

os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"]        = sys.executable

In [2]:
from pyspark import SparkContext
from sympy import *
from drudge import *
#from gristmill import *
print("finished importing")

finished importing


In [3]:
import re

# ----------------------------------------------------------------------------
# 1) Start Spark & BCS quasiparticle drudge
# ----------------------------------------------------------------------------
ctx = SparkContext('local[*]', 'bcs_ccsd')
dr  = ReducedBCSDrudge(ctx)
dr.full_simplify = True
print("finished")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/27 14:54:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/27 14:54:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
                                                                                

finished


In [6]:
P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N

p,q,r,s,i,j,k,l = dr.names.A_dumms[:8]
#p,q,r,s,i,j,k,l = symbols('p q r s i j k l')
 


Lmax = symbols('Lmax', integer=True, positive=True)
L = Range('L', 1, Lmax)  # symbolic chain 1..Lmax (boundary handling is symbolic)
dr.set_dumms(L, (p,))
G =  Symbol(r'\Delta')
#expr = Rational(1,4)*(Pdag[p] + P[p]) - Rational(1,2)*(N[p-1]-1)*(Pdag[p] + P[p])*(N[p+1] - 1) +Rational(1,4)*G*(N[p-1] - 1)*(N[p+1] - 1)
expr = (Pdag[p] + P[p])/4 - (N[p-1]-1)*(Pdag[p] + P[p])*(N[p+1] - 1)/4 + G*(N[p-1] - 1)*(N[p+1] - 1)/4
expr = dr.sum((p,L),expr).merge()
expr.display()



<IPython.core.display.Math object>

In [5]:
 
u, v = IndexedBase('u'), IndexedBase('v')
E0 = Symbol(r'E_0')
H120, H110 , H021 = IndexedBase('h120'), IndexedBase('h110'), IndexedBase('h021')
H011, H110 = IndexedBase('h011'), IndexedBase('h110')
H100, H011 = IndexedBase('h100'), IndexedBase('h011')
H001, H020 = IndexedBase('h001'), IndexedBase('h020')
H010 = IndexedBase('h010') 
H000 = IndexedBase('h000')

P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N
P_i_dag = (u[p]*v[p]+u[p]**2*Pdag[p]
     - u[p]*v[p]*N[p]
     - v[p]**2*P[p])

P_j_dag = (
        u[q]*v[q]
      + u[q]**2 * Pdag[q]
      - u[q]*v[q] * N[q]
      - v[q]**2 * P[q]
    )

P_k_dag = (
        u[r]*v[r]
      + u[r]**2 * Pdag[r]
      - u[r]*v[r] * N[r]
      - v[r]**2 * P[r]
    )
P_i = (
        (u[p])*(v[p])
      + (u[p])**2 * P[p]
      - (u[p])*(v[p]) * N[p]
      - (v[p])**2 * Pdag[p]
    )
P_j = ((u[q])*(v[q])
      + (u[q])**2 * P[q]
      - (u[q])*(v[q]) * N[q]
      - (v[q])**2 * Pdag[q])

P_k = ((u[r])*(v[r])
      + (u[r])**2 * P[r]
      - (u[r])*(v[r]) * N[r]
      - (v[r])**2 * Pdag[r])
#hc = P_j_dag * P_i
N_i = 2*(v[p])*v[p] +((u[p])*u[p] - (v[p])*v[p])*N[p] +2*u[p]*(v[p])*Pdag[p]\
            +2*(u[p])*v[p]*P[p]
N_j = 2*(v[q])*v[q] +((u[q])*u[q] - (v[q])*v[q])*N[q] +2*u[q]*(v[q])*Pdag[q]\
            +2*(u[q])*v[q]*P[q]

N_k = 2*(v[r])*v[r] +((u[r])*u[r] - (v[r])*v[r])*N[r] +2*u[r]*(v[r])*Pdag[r]\
       +2*(u[r])*v[r]*P[r]


In [7]:
dr.set_symm(H021, Perm([1, 0,2],IDENT), valence=3) #This is correct, the symmetry factor is already 
dr.set_symm(H120, Perm([0, 2,1],IDENT), valence=3)
dr.set_symm(H020, Perm([1,0], IDENT),valence=2)
expr = H120[p,q,r]*P_i_dag*N_j*N_k  + H021[p,q,r]*N_i*N_j*P_k  + H110[p,q]*P_i_dag*N_j\
 + H100[p]*P_i_dag + H011[p,q]*N_i*P_j + H001[p]*P_i + H010[p]*N_i + H020[p,q]*N_i*N_j
H_N =  dr.einst(expr)
H_N = H_N.simplify().merge()
H_N.display()


<IPython.core.display.Math object>

In [8]:
X = u[p]*H120[p,q]*P_i_dag
X = dr.einst(X).simplify().merge()

In [9]:
from drudge import Term

def coeff_with_internal_sums(ts):
    """
    Strip the operator word and *only* those sums that run over
    indices appearing in the operator word. Keep any other internal sums. We need this for Integrals.
    """
    def proc(t):
        # Collect indices used in the operator word
        op_inds = set()
        for v in t.vecs:
            # depending on your Drudge version, this may be v.indices or v.inds
            for ind in v.indices:
                op_inds.add(ind)

        # Keep only sums whose index is NOT in the operator word
        new_sums = tuple((i, base) for (i, base) in t.sums if i not in op_inds)

        # Strip vecs; keep amp unchanged
        return Term(new_sums, t.amp, ())

    return ts.map(proc)


In [21]:
def coeff_only(ts):
    """Return a TermSum with the same sums/amp but with vecs removed."""
    # ts can be a TermSum or anything with .map over terms
    return ts.map(lambda t: Term(t.sums, t.amp, ()))   # vecs = ()

In [10]:
# Loop through each local term in H_N
for i, term in enumerate(H_N.local_terms):
    print(f"--- Term {i} ---")
       # The scalar coefficient
    print("Operators (vecs):", term.vecs)   # The associated vector operators
    print()

--- Term 0 ---
Operators (vecs): ()

--- Term 1 ---
Operators (vecs): (Vec('N', (p)),)

--- Term 2 ---
Operators (vecs): (Vec('P^\\dagger', (p)), Vec('P', (q)), Vec('P', (r)))

--- Term 3 ---
Operators (vecs): (Vec('P^\\dagger', (p)), Vec('P^\\dagger', (q)))

--- Term 4 ---
Operators (vecs): (Vec('P^\\dagger', (p)), Vec('N', (q)), Vec('N', (r)))

--- Term 5 ---
Operators (vecs): (Vec('P^\\dagger', (p)), Vec('P^\\dagger', (q)))

--- Term 6 ---
Operators (vecs): (Vec('P^\\dagger', (p)),)

--- Term 7 ---
Operators (vecs): (Vec('P^\\dagger', (p)),)

--- Term 8 ---
Operators (vecs): (Vec('P^\\dagger', (p)), Vec('P^\\dagger', (q)), Vec('P', (r)))

--- Term 9 ---
Operators (vecs): (Vec('N', (p)), Vec('P', (q)))

--- Term 10 ---
Operators (vecs): (Vec('P', (p)),)

--- Term 11 ---
Operators (vecs): (Vec('P', (p)),)

--- Term 12 ---
Operators (vecs): (Vec('N', (p)), Vec('P', (q)))

--- Term 13 ---
Operators (vecs): (Vec('N', (p)), Vec('N', (q)), Vec('P', (r)))

--- Term 14 ---
Operators (vecs): 

In [11]:
def get_PdagPdagP(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
    # Normal-ordered triple: P† P† P
    return labels == ['P^\\dagger', 'P^\\dagger','P']

def get_PdagPP(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger', 'P','P']

def get_NNN(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['N', 'N','N']


def get_PPP(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P', 'P','P']


def get_PdagPdagN(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger', 'P^\\dagger','N']


def get_NNP(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['N', 'N','P']


def get_NPP(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['N', 'P','P']


def get_PdagPdagPdag(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger','P^\\dagger','P^\\dagger']


def get_PdagNN(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger', 'N','N']


def get_PdagNP(term):
    vecs = term.vecs
    # Must be exactly 3 operators
    if len(vecs) != 3:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger', 'N','P']

In [12]:
def get_Pdag(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 1:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger']

def get_P(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 1:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P']

def get_N(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 1:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['N']


In [13]:
def get_NN(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 2:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['N','N']

def get_NP(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 2:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['N','P']

def get_PP(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 2:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P','P']

def get_PdagPdag(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 2:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger','P^\\dagger']

def get_PdagN(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 2:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger','N']

def get_PdagP(term):
    vecs = term.vecs
    # Must be exactly 1 operators
    if len(vecs) != 2:
        return False
    labels = [v.label for v in vecs]
     
    return labels == ['P^\\dagger','P']


In [14]:
tensor0_ = H_N.filter(lambda term: len(term.vecs) == 0)
#tensor0_ = dr.einst(tensor0_).simplify().merge().merge()
tensor0_ = tensor0_.simplify().merge().merge()
 
tensor0_.display()

<IPython.core.display.Math object>

In [18]:
tensor1_ = H_N.filter(lambda term: len(term.vecs) == 1)

#tensor1_ = dr.einst(tensor1_).simplify().merge().merge()
tensor1_ = tensor1_.simplify().merge().merge()
tensor1_.display()


<IPython.core.display.Math object>

In [19]:
tensor_ = H_N.filter(lambda term: len(term.vecs) == 2)
tensor_ = dr.einst(tensor_).simplify().merge().merge()
tensor_.display()

<IPython.core.display.Math object>

In [25]:
tensor3_ = H_N.filter(lambda term: len(term.vecs) == 3)
#tensor3_ = tensor3_.simplify().merge().merge()
tensor3_ = dr.einst(tensor3_).simplify().merge().merge()

tensor3_.display()

<IPython.core.display.Math object>

In [26]:
pdagpdagp_terms = tensor3_.terms.filter(get_PdagPdagP)
#H201_dr = Tensor(dr,  coeff_with_internal_sums(pdagpdagp_terms)).simplify().merge()
H201_dr = Tensor(dr,   coeff_only(pdagpdagp_terms)).simplify().merge()

 

pdagpdagpdag_terms = tensor3_.terms.filter(get_PdagPdagPdag)
H300_dr = Tensor(dr,  coeff_with_internal_sums(pdagpdagpdag_terms)).simplify().merge()
 

pdagpp_terms = tensor3_.terms.filter(get_PdagPP)
H102_dr = Tensor(dr,  coeff_with_internal_sums(pdagpp_terms)).simplify().merge()

ppp_terms = tensor3_.terms.filter(get_PPP)
H003_dr = Tensor(dr,  coeff_with_internal_sums(ppp_terms)).simplify().merge()

nnn_terms = tensor3_.terms.filter(get_NNN)
H030_dr = Tensor(dr,  coeff_with_internal_sums(nnn_terms)).simplify().merge()

pdagpdagn_terms = tensor3_.terms.filter(get_PdagPdagN)
H210_dr = Tensor(dr,  coeff_with_internal_sums(pdagpdagn_terms)).simplify().merge()

npp_terms = tensor3_.terms.filter(get_NPP)
H012_dr = Tensor(dr,  coeff_with_internal_sums(npp_terms)).simplify().merge()

pdagnn_terms = tensor3_.terms.filter(get_PdagNN)
H120_dr = Tensor(dr,  coeff_with_internal_sums(pdagnn_terms)).simplify().merge()

nnp_terms = tensor3_.terms.filter(get_NNP)
H021_dr = Tensor(dr,  coeff_with_internal_sums(nnp_terms)).simplify().merge()

pdagnp_terms = tensor3_.terms.filter(get_PdagNP)
H111_dr = Tensor(dr,  coeff_with_internal_sums(pdagnp_terms)).simplify().merge()

# rank-1 vecs

pdag_terms = tensor1_.terms.filter(get_Pdag)
H100_dr = Tensor(dr, coeff_with_internal_sums(pdag_terms)).simplify().merge()

n_terms = tensor1_.terms.filter(get_N)
H010_dr = Tensor(dr, coeff_with_internal_sums(n_terms)).simplify().merge()

p_terms = tensor1_.terms.filter(get_P)
H001_dr = Tensor(dr, coeff_with_internal_sums(p_terms)).simplify().merge()

#rank-2 vecs

pdagpdag_terms = tensor_.terms.filter(get_PdagPdag)
H200_dr = Tensor(dr, coeff_with_internal_sums(pdagpdag_terms)).simplify().merge()

pdagp_terms = tensor_.terms.filter(get_PdagP)
H101_dr = Tensor(dr, coeff_with_internal_sums(pdagp_terms)).simplify().merge()

nn_terms = tensor_.terms.filter(get_NN)
H020_dr = Tensor(dr, coeff_with_internal_sums(nn_terms)).simplify().merge()

pp_terms = tensor_.terms.filter(get_PP)
H002_dr = Tensor(dr, coeff_with_internal_sums(pp_terms)).simplify().merge()

pdagn_terms = tensor_.terms.filter(get_PdagN)
H110_dr = Tensor(dr, coeff_with_internal_sums(pdagn_terms)).simplify().merge()

np_terms = tensor_.terms.filter(get_NP)
H011_dr = Tensor(dr,  coeff_with_internal_sums(np_terms)).simplify().merge()

#scalar
tensor0_ = H_N.filter(lambda term: len(term.vecs) == 0)
tensor0_ = dr.einst(tensor0_).simplify().merge()
H000_dr =  tensor0_
H001_dr.display() 

<IPython.core.display.Math object>

In [ ]:
import re
from sympy.printing.fortran import FCodePrinter
from drudge import Tensor
from gristmill import FortranPrinter, optimize, get_flop_cost
from sympy import IndexedBase

H120, H110 = IndexedBase('W120'), IndexedBase('W110') 
H011,H003, H300, H030 =   IndexedBase('W011'), IndexedBase('W003'), IndexedBase('W300'), IndexedBase('W030')
H020 = IndexedBase('W020')
H002, H200 = IndexedBase('W002'), IndexedBase('W200')
H000 = IndexedBase('W000')
H101 = IndexedBase('W101')
H010 , H001, H100 = IndexedBase('W010'),IndexedBase('W001'), IndexedBase('W100')
H201, H210, H102 = IndexedBase('W201'),IndexedBase('W210'),IndexedBase('W102')
H021, H012, H111 = IndexedBase('W021'),IndexedBase('W012'),IndexedBase('W111') 


work_eqn = []

 
work_eqn.append(dr.define(H000, H000_dr))
work_eqn.append(dr.define(H100[p], H100_dr))
work_eqn.append(dr.define(H001[p], H001_dr)) 
work_eqn.append(dr.define(H010[p], H010_dr))
work_eqn.append(dr.define(H002[p,q], H002_dr))
work_eqn.append(dr.define(H200[p,q], H200_dr)) 
work_eqn.append(dr.define(H101[p,q], H101_dr))
work_eqn.append(dr.define(H110[p,q], H110_dr)) 
work_eqn.append(dr.define(H011[p,q], H011_dr))
work_eqn.append(dr.define(H020[p,q], H020_dr))
work_eqn.append(dr.define(H120[p,q,r], H120_dr))
work_eqn.append(dr.define(H201[p,q,r], H201_dr))
work_eqn.append(dr.define(H012[p,q,r], H012_dr))
work_eqn.append(dr.define(H210[p,q,r], H210_dr))
work_eqn.append(dr.define(H021[p,q,r], H021_dr))
work_eqn.append(dr.define(H102[p,q,r], H102_dr))
work_eqn.append(dr.define(H111[p,q,r], H111_dr))
work_eqn.append(dr.define(H003[p,q,r], H003_dr))
work_eqn.append(dr.define(H030[p,q,r], H030_dr))
work_eqn.append(dr.define(H300[p,q,r], H300_dr))


print("Original cost:", get_flop_cost(work_eqn, leading=True))

# Optimize (optional)

'''def show_lhs_types(defs, title):
    print(f"\n=== LHS type check ({title}) ===")
    bad = []
    for k, d in enumerate(defs):
        base = d.base
        is_sympy = isinstance(base, (Symbol, IndexedBase))
        print(f"{k:03d}: base={base!r}  type={type(base).__name__}  exts={getattr(d, 'exts', None)}  sympy? {is_sympy}")
        if not is_sympy:
            bad.append((k, d))
    return bad'''

# if you already did `eval_seq = optimize(work_eqn, ...)`:
# check BOTH the original and optimized sequences
 
 
eval_eq = optimize(work_eqn, interm_fmt='tau{}',drop_cutoff=2)
 

 
print("Original cost:", get_flop_cost(eval_eq))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_eq))
 
evals = fort.doprint(eval_eq)
with open('SFS_1D_Integrals.f90', 'w') as fp:
    fp.write(evals)

print("Printed!")
 

 

 
 